In [ ]:
import json

def best_n_docs_per_query(n=1000):
    '''
    Get the best n docs per query using tfidf
    '''
    
    best_docs_per_query = defaultdict(list)
    queries_done = 0
    duplicates = set()
    for query_id, query_term_ids in tokenized_queries.items():
        document_ids = itertools.chain(*[list(inverted_index[i].keys()) for i in query_term_ids])
        scores = []
        for doc_id in document_ids:
            if doc_id in duplicates:
                continue
            score = sum(tfidf(doc_id, query_term_id) for query_term_id in query_term_ids)
            scores.append((doc_id, str(score)))
            duplicates.add(doc_id)
        best_docs_per_query[query_id] = sorted(scores, key=lambda x: float(x[1]), reverse=True)[:n]
        queries_done += 1
        print(queries_done, "queries run on all the documents")
        
    with open('best_'+ str(n) +'_docs_per_query_no_duplicates.json', 'w') as f:
        json.dump(best_docs_per_query, f) 
            
    return best_docs_per_query

## Create positional index with for every document all query term positions
positional_index = {}
for int_doc_id in range(index.document_base(), index.maximum_document()):
    positional_index[int_doc_id] = defaultdict(list)
    doc = index.document(int_doc_id)
    for i, word_id in enumerate(doc[1]):
        if word_id in query_term_ids:
            positional_index[int_doc_id][word_id].append(i)

with open('positional_index.json', 'w') as f:
    json.dump(positional_index, f)            

## Precompute all gaussian kernel values
maximum = max(document_lengths.values())
gaussian_kernel = {}
maximum = max(document_lengths.values())
for j in range(0, maximum+1):
    gaussian_kernel[j] = {}
    for i in range(0, maximum+1):
         gaussian_kernel[j][i] = gaussian(j,i)
            
with open('gaussian_kernel.json', 'w') as f:
        json.dump(gaussian_kernel, f)
            
## Precompute all gaussian cdf kernel values
gaussian_cdf_kernel = {}
for N in range(0, maximum+1):
    gaussian_cdf_kernel[N] = {}
    for i in range(0, maximum+1):
        gaussian_cdf_kernel[N][i] = gaussian_cdf(N,i)
    
with open('gaussian_cdf_kernel.json', 'w') as f:
        json.dump(gaussian_cdf_kernel, f)